In [1]:
#train the baseline CNN model for MNIST. Model is identical to that used in the tensorflow
#tutorial: https://www.tensorflow.org/versions/master/tutorials/mnist/pros/index.html

import mnist_input
import mnist_model
reload(mnist_model)
import numpy as np
import tensorflow as tf
mnist = mnist_input.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
checkpoint_path = "/home/justin/Programming/AdverserialMNIST/saved_models/baseline_40epochs.ckpt"
x, y_ = mnist_model.place_holders()
y_conv, keep_prob, variable_dict = mnist_model.model(x)

In [4]:
cross_entropy = -tf.reduce_sum(y_*tf.log(tf.clip_by_value(y_conv,1e-10,1.0))) #avoid 0*log(0) error

train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

saver = tf.train.Saver(variable_dict)
sess = tf.InteractiveSession()
sess.run(tf.initialize_all_variables())
saver.restore(sess, checkpoint_path)

Exception AssertionError: AssertionError() in <bound method InteractiveSession.__del__ of <tensorflow.python.client.session.InteractiveSession object at 0x7f371dea52d0>> ignored


In [5]:
#define the l1 norm of all model parameters
absolute_sums = []
for variable in variable_dict.values():
    absolute_sums.append(tf.reduce_sum(tf.abs(variable)))
l1_sum = tf.add_n(absolute_sums)

In [ ]:
#compute accuracy on test set
#my GPU gives a memory error if I try to run accuracy on the whole test set
#so we feed in the test set in batches and aggregate the results
def print_test_accuracy():
    idx = 0
    batch_size = 500
    num_correct = 0
    while(idx < len(mnist.test.images)):
        num_correct += np.sum(correct_prediction.eval(feed_dict = {
               x: mnist.test.images[idx:idx+batch_size], 
               y_: mnist.test.labels[idx:idx+batch_size], keep_prob: 1.0
                    }))
        idx+=batch_size
    print "test accuracy: %f" %(float(num_correct)/float(len(mnist.test.images)))


In [8]:
for i in range(70000):
  batch = mnist.train.next_batch(50)
  if i%1000 == 0: #every 1000 batches we save the model and output the train accuracy
    print_test_accuracy()
    saver.save(sess, checkpoint_path)
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    l1_norm = sess.run(l1_sum)
    print("step %d, training accuracy %g, l1_sum: %f"%(i, train_accuracy, l1_norm))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})




test accuracy: 0.993000
step 0, training accuracy 1, l1_sum: 241367.609375
test accuracy: 0.993100
step 1000, training accuracy 1, l1_sum: 241550.531250
test accuracy: 0.993100
step 2000, training accuracy 1, l1_sum: 241670.468750
test accuracy: 0.993200
step 3000, training accuracy 1, l1_sum: 241804.812500
test accuracy: 0.993800
step 4000, training accuracy 1, l1_sum: 241941.015625
test accuracy: 0.993900
step 5000, training accuracy 1, l1_sum: 242055.953125
test accuracy: 0.994200
step 6000, training accuracy 1, l1_sum: 242186.765625
test accuracy: 0.993500
step 7000, training accuracy 1, l1_sum: 242325.500000
test accuracy: 0.992500
step 8000, training accuracy 1, l1_sum: 242432.656250
test accuracy: 0.993200
step 9000, training accuracy 1, l1_sum: 242538.328125
test accuracy: 0.992500
step 10000, training accuracy 1, l1_sum: 242670.703125
test accuracy: 0.993500
step 11000, training accuracy 1, l1_sum: 242796.546875
test accuracy: 0.993100
step 12000, training accuracy 1, l1_sum: 

In [6]:
#total weight of all of the parameters in the model (for comparison with the l1-regularized model)
sess.run(l1_sum)

239372.95